# midterm 2 practice


In [99]:
FILEIN = "data/midterm_2_data.xlsx"
STOCKS_SHEET = "sector excess returns"
FACTORS_SHEET = "factors excess returns"

stocks_excess_returns = pmh.read_excel_default(FILEIN, sheet_name=STOCKS_SHEET)
factors_excess_returns = pmh.read_excel_default(FILEIN, sheet_name=FACTORS_SHEET)

AQR_FACTORS = ['MKT', 'RMW', 'HML', 'UMD']

COV_MATRIX = 1/2
TESTING_FACTOR_PREMIA = False

USE_INTERCEPT = False
TRAINING_START = "2018"
TRAINING_END = "2022"
TESTING_START = "2023"
TESTING_END = "2024"

SAMPLE_ETFS = ['Agric', 'Food', 'Soda']

aqr_factors_excess_returns = factors_excess_returns[AQR_FACTORS]
training_stocks_excess_returns = stocks_excess_returns.loc[TRAINING_START:TRAINING_END]
training_aqr_factors_excess_returns = aqr_factors_excess_returns.loc[TRAINING_START:TRAINING_END]

testing_stocks_excess_returns = stocks_excess_returns.loc[TESTING_START:TESTING_END]
testing_aqr_factors_excess_returns = aqr_factors_excess_returns.loc[TESTING_START:TESTING_END]

if TESTING_FACTOR_PREMIA:
    factor_premia_returns = testing_aqr_factors_excess_returns.copy()
else:
    factor_premia_returns = training_aqr_factors_excess_returns.copy()

iterative_factor_regressions = (
    pmh.calc_iterative_regression(
        training_stocks_excess_returns,
        training_aqr_factors_excess_returns,
        intercept=USE_INTERCEPT,
        keep_columns=["Beta", "Alpha"],
        drop_columns=["Annualized Alpha"],
        warnings=False
    )
    .reset_index()
    .assign(index=lambda df: df['index'].map(lambda x: x.replace(' no Intercept', '')))
    .rename({'index': None}, axis=1)
    .set_index(None)
)

premia_and_expected_excess_returns = (
    iterative_factor_regressions
    .assign(**{
        f"{factor} Premia": (lambda df, factor=factor: df[f"{factor} Beta"] * factor_premia_returns[f"{factor}"].mean())
        for factor in AQR_FACTORS
    })
    .assign(**{"Expected Return": lambda df: df[
        [f + " Premia" for f in AQR_FACTORS]
    ].sum(axis=1)})
    .drop('Alpha', axis=1)
    .assign(**{
        f"Annualized {premia}": (lambda df, premia=premia: df[f"{premia}"] * 12)
        for premia in [f + " Premia" for f in AQR_FACTORS] + ["Expected Return"]
    })
)

(
    premia_and_expected_excess_returns
    .iloc[:3]
    .loc[:, ['Expected Return', 'Annualized Expected Return']]
)
stocks_expected_excess_return = premia_and_expected_excess_returns['Expected Return']
display(stocks_expected_excess_return.head())
display(training_stocks_excess_returns.head())
training_tangency_weights_expected_returns = pmh.calc_tangency_weights(
    training_stocks_excess_returns,
    cov_mat=COV_MATRIX,
    expected_returns=stocks_expected_excess_return,
    name="AQR Expected Returns"
)
display(training_tangency_weights_expected_returns.iloc[:3])


Agric   0.0037
Food    0.0055
Soda    0.0073
Beer    0.0071
Smoke   0.0051
Name: Expected Return, dtype: float64

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.0421,0.0027,0.0564,-0.0023,0.0001,0.0368,0.1817,0.0512,-0.0243,0.0703,...,0.0298,0.0241,0.0469,0.1143,0.0240,0.0716,0.0390,0.0360,0.0512,0.0386
2018-02-01,0.0110,-0.0689,-0.0849,-0.0685,-0.0672,0.0056,0.0065,-0.0353,-0.0678,-0.0202,...,-0.0383,-0.0639,-0.0549,-0.0456,-0.0418,-0.0228,-0.0453,-0.0348,-0.0145,-0.0537
2018-03-01,-0.0506,-0.0239,-0.0052,0.0113,-0.0176,-0.0935,0.0028,-0.0024,0.0176,0.0057,...,-0.0166,0.0047,-0.0117,-0.0341,-0.0038,-0.0475,-0.0224,0.0442,-0.0141,-0.0360
2018-04-01,0.0784,-0.0358,-0.0093,-0.0448,-0.1431,0.0630,0.0288,-0.0062,-0.0573,0.0315,...,-0.0152,0.0013,0.0069,0.0398,0.0354,0.0085,0.0337,-0.0130,-0.0169,-0.0137
2018-05-01,0.0181,-0.0097,-0.0141,-0.0137,-0.0201,0.0069,0.0938,0.0059,-0.0049,0.0375,...,-0.0321,0.0478,0.0142,0.0116,-0.0220,0.0091,-0.0121,-0.0340,0.0021,-0.0014


,AQR Expected Returns Weights
Agric,-0.0307
Food,0.0153
Soda,0.1329


In [100]:
import numpy as np
import os
from arch import arch_model
import re
import sys
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
import sys
from typing import Tuple, Dict, Union, Callable
notebook_dir = "practice"
if os.path.basename(os.getcwd()) == notebook_dir:
    os.chdir(os.pardir)  # Change to parent directory
    sys.path.append(os.getcwd())  # Add the new current directory to sys.path

import cmds.portfolio_management_helper as pmh
import functions.functions_andy as fa
print(os.getcwd())

from sklearn.linear_model import LinearRegression
from dask.distributed import Client, wait
# From HW1
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.metrics import r2_score
from scipy.stats import describe

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

c:\Users\Andik\OneDrive\Desktop\Chicago\Portfolio theory and risk management\githubrepo\finm-portfolio-2024


In [101]:
factor_data = pmh.read_excel_default(excel_name="./data/midterm_2_data.xlsx", sheet_name=1)
sector_data = pmh.read_excel_default(excel_name="./data/midterm_2_data.xlsx", sheet_name=0)
factor_data.index = pd.to_datetime(factor_data.index)
sector_data.index = pd.to_datetime(sector_data.index)
display(factor_data.head())
display(sector_data.head())

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.0551,0.0175,-0.0170,0.0755
1980-02-01,-0.0122,0.0061,0.0004,0.0788
1980-03-01,-0.1290,-0.0101,0.0146,-0.0955
1980-04-01,0.0397,0.0106,-0.0210,-0.0043
1980-05-01,0.0526,0.0038,0.0034,-0.0112


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,-0.0076,0.0285,0.0084,0.1009,-0.0143,0.1002,0.0362,0.0323,0.0048,0.0059,...,0.0158,0.0875,0.0465,-0.0126,0.0430,-0.0283,0.0258,0.0768,0.0308,0.0669
1980-02-01,0.0105,-0.0608,-0.0966,-0.0322,-0.0569,-0.0323,-0.0521,-0.0800,-0.0555,-0.0167,...,-0.0079,-0.0541,-0.0346,-0.0639,-0.0652,-0.0854,-0.0959,-0.0347,-0.0282,-0.0274
1980-03-01,-0.2224,-0.1119,-0.0167,-0.1469,-0.0193,-0.1271,-0.0826,-0.1237,-0.0566,-0.0668,...,-0.0819,-0.1509,-0.1098,-0.0906,-0.1449,-0.0560,-0.0880,-0.2451,-0.1254,-0.1726
1980-04-01,0.0449,0.0766,0.0232,0.0321,0.0830,-0.0529,0.0783,0.0153,0.0304,0.0115,...,0.0420,-0.0103,-0.0312,0.0353,0.0542,0.0728,0.0530,0.0977,0.0447,0.0769
1980-05-01,0.0632,0.0793,0.0457,0.0863,0.0815,0.0509,0.0324,0.0886,0.0560,0.0098,...,0.0564,0.1063,0.1142,0.0877,0.1134,0.0578,0.0557,0.0915,0.0844,0.0685


In [102]:
train_sector_data = sector_data.loc["2018":"2022",:]
train_factor_data = factor_data.loc["2018":"2022",:]
results = {}
for column in train_sector_data.columns:
    y = train_sector_data.loc[:,[column]]
    X = train_factor_data
    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    beta = model.coef_[0]
    results[column] = beta
# cross sectional regression
beta_df = pd.DataFrame(results, index=X.columns).T
    

In [ ]:

factor_premia = train_factor_data.mean().to_frame("Factor_Premia")
display(factor_premia)
model_implied_premia = beta_df @ factor_premia
# display(model_implied_premia)
model_implied_data = pd.concat([model_implied_premia,beta_df],axis=1)
display(model_implied_data)
print("Model Implied Data for Agric, Food, and Soda sector")
display(model_implied_data.iloc[0:3,1:])


,Factor_Premia
MKT,0.0075
HML,-0.0003
RMW,0.0051
UMD,0.0039


,Factor_Premia,MKT,HML,RMW,UMD
Agric,0.0037,0.8324,0.5565,-0.5021,0.0390
Food,0.0055,0.5245,0.2055,0.3097,-0.0036
Soda,0.0073,0.5402,0.1791,0.6384,0.0137
Beer,0.0071,0.5413,0.0230,0.6297,-0.0405
Smoke,0.0051,0.4982,0.4431,0.4022,-0.1340
Toys,0.0059,1.4060,-0.1204,-1.0136,0.1251
Fun,0.0057,1.3699,-0.1865,-0.8659,-0.0544
Books,0.0043,1.0783,0.2794,-0.4122,-0.4108
Hshld,0.0077,0.4769,-0.0946,0.8442,-0.0765
Clths,0.0095,1.1314,-0.1716,0.3427,-0.2098


Model Implied Data for Agric, Food, and Soda sector


,MKT,HML,RMW,UMD
Agric,0.8324,0.5565,-0.5021,0.0390
Food,0.5245,0.2055,0.3097,-0.0036
Soda,0.5402,0.1791,0.6384,0.0137


In [104]:
# regularized cov matrix 
sector_mean = model_implied_data.loc[:,["Factor_Premia"]]
sector_cov = train_sector_data.cov()
reg_cov = (sector_cov + np.diag(np.diag(sector_cov))) / 2
inv_reg_cov = np.linalg.inv(reg_cov)
weight = inv_reg_cov @ sector_mean / (np.ones((1,inv_reg_cov.shape[0])) @ inv_reg_cov @ sector_mean).values[0,0]
weight.index = train_sector_data.columns
weight.columns = ["Weight"]
display(weight.T)
# portfolio performance
portfolio_mean = (sector_mean.T @ weight).values[0,0]
portfolio_std = np.sqrt((weight.T @ sector_cov @ weight).values[0,0])
portfolio_sharpe = portfolio_mean / portfolio_std
print("Portfolio Performance")
print(f"Portfolio Mean: {portfolio_mean*12}")
print(f"Portfolio Std: {portfolio_std*12**0.5}")
print(f"Portfolio Sharpe: {portfolio_sharpe*12**0.5}")
#OOS
test_sector_data = sector_data.loc["2023":"2024",:]
test_factor_data = factor_data.loc["2023":"2024",:]
oos_portfolio_mean = (test_sector_data.mean() @ weight).values[0]
oos_port_std = np.sqrt((weight.T @ test_sector_data.cov() @ weight).values[0])
oos_port_sharpe = oos_portfolio_mean / oos_port_std
print("OOS Portfolio Performance")
print(f"OOS Portfolio Mean: {oos_portfolio_mean*12}")
print(f"OOS Portfolio Std: {oos_port_std*12**0.5}")
print(f"OOS Portfolio Sharpe: {oos_port_sharpe*12**0.5}")
pmh.calc_tangency_weights(train_sector_data, cov_mat=0.5, expected_returns=sector_mean["Factor_Premia"], name="Sector Expected Returns").iloc[:3]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Weight,-0.0307,0.0153,0.1329,0.0794,0.0123,-0.0412,-0.0318,-0.0920,0.1536,0.0432,...,-0.0045,-0.0202,0.0553,0.0603,0.0720,-0.0278,0.0528,-0.0729,0.0202,0.0240


Portfolio Performance
Portfolio Mean: 0.13073456077733986
Portfolio Std: 0.1578017363111297
Portfolio Sharpe: 0.8284735252822387
OOS Portfolio Performance
OOS Portfolio Mean: 0.18117551805196944
OOS Portfolio Std: [0.1195488]
OOS Portfolio Sharpe: [1.51549422]


,Sector Expected Returns Weights
Agric,-0.0307
Food,0.0153
Soda,0.1329


In [108]:
# using sample averages
model_implied_data["Factor_Premia"] = train_sector_data.mean()
# regularized cov matrix 
sector_mean = model_implied_data.loc[:,["Factor_Premia"]]
sector_cov = train_sector_data.cov()
reg_cov = (sector_cov + np.diag(np.diag(sector_cov))) / 2
inv_reg_cov = np.linalg.inv(reg_cov)
weight = inv_reg_cov @ sector_mean / (np.ones((1,inv_reg_cov.shape[0])) @ inv_reg_cov @ sector_mean).values[0,0]
weight.index = train_sector_data.columns
weight.columns = ["Weight"]
display(weight.T)
# portfolio performance
portfolio_mean = (sector_mean.T @ weight).values[0,0]
portfolio_std = np.sqrt((weight.T @ sector_cov @ weight).values[0,0])
portfolio_sharpe = portfolio_mean / portfolio_std
print("Portfolio Performance")
print(f"Portfolio Mean: {portfolio_mean*12}")
print(f"Portfolio Std: {portfolio_std*12**0.5}")
print(f"Portfolio Sharpe: {portfolio_sharpe*12**0.5}")
#OOS
test_sector_data = sector_data.loc["2023":"2024",:]
test_factor_data = factor_data.loc["2023":"2024",:]
oos_portfolio_mean = (test_sector_data.mean() @ weight).values[0]
oos_port_std = np.sqrt((weight.T @ test_sector_data.cov() @ weight).values[0])
oos_port_sharpe = oos_portfolio_mean / oos_port_std
print("OOS Portfolio Performance")
print(f"OOS Portfolio Mean: {oos_portfolio_mean*12}")
print(f"OOS Portfolio Std: {oos_port_std*12**0.5}")
print(f"OOS Portfolio Sharpe: {oos_port_sharpe*12**0.5}")
pmh.calc_tangency_weights(train_sector_data, cov_mat=0.5, name="Sector Expected Returns").iloc[:3]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Weight,0.1441,-0.0698,0.3227,0.1076,-0.0434,-0.1396,-0.0765,-0.1299,0.2666,0.0749,...,-0.1024,-0.1459,0.1907,0.0044,0.0831,-0.0162,0.3306,-0.0479,0.0930,0.0078


Portfolio Performance
Portfolio Mean: 0.40712561571510963
Portfolio Std: 0.17726660032568245
Portfolio Sharpe: 2.29668541601813
OOS Portfolio Performance
OOS Portfolio Mean: 0.17680083032564703
OOS Portfolio Std: [0.15300974]
OOS Portfolio Sharpe: [1.15548741]


,Sector Expected Returns Weights
Agric,0.1441
Food,-0.0698
Soda,0.3227


In [119]:
train_factor_data = factor_data.loc["2018":"2022",:]
test_factor_data = factor_data.loc["2023":"2024",:]
weights =pmh.calc_tangency_weights(train_factor_data, cov_mat=0.5, name="Factor Expected Returns")
oos_port_mean = (test_factor_data @ weights).mean()
oos_port_std = np.sqrt(weights.T @ test_factor_data.cov() @ weights)
oos_port_sharpe = oos_port_mean / oos_port_std
print("OOS Portfolio Performance")
print(f"OOS Portfolio Mean: {oos_port_mean.values*12}")
print(f"OOS Portfolio Std: {oos_port_std.values*12**0.5}")
print(f"OOS Portfolio Sharpe: {oos_port_sharpe.values*12**0.5}")



OOS Portfolio Performance
OOS Portfolio Mean: [0.06237596]
OOS Portfolio Std: [[0.05819096]]
OOS Portfolio Sharpe: [[1.07191825]]


In [123]:
# Q3
## Long returns
factor_log_returns = factor_data.apply(np.log1p)

display(pmh.calc_summary_statistics(factor_log_returns).loc[:,["Annualized Mean","Annualized Vol", "Annualized Sharpe"]])

# print(factor_log_returns.mean()*12)
# print(factor_log_returns.std()*12**0.5)
# print((factor_log_returns.mean()*12)/(factor_log_returns.std()*12**0.5))

Assuming monthly returns with annualization term of 12
Assuming excess returns were provided to calculate Sharpe. If returns were provided (steady of excess returns), the column "Sharpe" is actually "Mean/Volatility"


,Annualized Mean,Annualized Vol,Annualized Sharpe
MKT,0.0735,0.1588,0.4630
HML,0.0198,0.1098,0.1801
RMW,0.0435,0.0836,0.5210
UMD,0.0501,0.1604,0.3122


In [126]:
# Q3.2
factor_log_cumulative_returns = factor_log_returns
mean = factor_log_cumulative_returns.mean()*12*15
std = factor_log_cumulative_returns.std()*12**0.5*np.sqrt(15)
sharpe = mean/std
display(pd.DataFrame({"Mean":mean,"Std":std,"Sharpe":sharpe}))

,Mean,Std,Sharpe
MKT,1.1032,0.6152,1.7933
HML,0.2965,0.4252,0.6975
RMW,0.6531,0.3237,2.0177
UMD,0.7514,0.6214,1.2093


In [132]:
# next single period
mean_single_period = factor_log_returns.mean()
std_single_period = factor_log_returns.std()
prob_single_period = norm.cdf(-mean_single_period/std_single_period)[-1]
print(f"Prob of negative return next period {prob_single_period}")
# next 15 years
mean_15_years = factor_log_returns.mean()*12*15
std_15_years = factor_log_returns.std()*12**0.5*np.sqrt(15)
prob_15_years = norm.cdf(-mean_15_years/std_15_years)[-1]
print(f"Prob of negative return next 15 years {prob_15_years}")

Prob of negative return next period 0.46408867614246724
Prob of negative return next 15 years 0.11326774683663715


In [152]:
factor_data_subset = factor_log_returns.loc["2009":,:]
X = (factor_data_subset["UMD"] - factor_data_subset["MKT"]).to_frame("UMD-MKT").dropna()
mean_single_period = X.mean()
std_single_period = X.std()
prob_single_period = 1 - norm.cdf(-mean_single_period/std_single_period)
print(f"Prob of negative return next period {prob_single_period}")
# next 15 years
mean_15_years = X.mean()*12*15
std_15_years = X.std()*12**0.5*np.sqrt(15)
prob_15_years = 1 - norm.cdf(-mean_15_years/std_15_years)
print(f"Prob of negative return next 15 years {prob_15_years}")


Prob of negative return next period [0.43450383]
Prob of negative return next 15 years [0.01346186]


In [140]:

factors = pd.read_excel(FILEIN, sheet_name=FACTORS_SHEET, index_col=0)
factors_log = np.log(factors + 1)
def prob(mu, sigma, h):
    return norm.cdf(np.sqrt(h)*mu/sigma)
umd_mkt_spread = factors_log['UMD'] - factors_log['MKT']
umd_mkt_spread = umd_mkt_spread.loc['2009':]

umd_mkt_mean = umd_mkt_spread.mean() * 12
umd_mkt_std = umd_mkt_spread.std() * np.sqrt(12)

prob_outperf_1_month = prob(umd_mkt_mean, umd_mkt_std, 1 / 12)
prob_outperf_15_year = prob(umd_mkt_mean, umd_mkt_std, 15)

print(f"Probability of UMD outperforming the market in a single month: {prob_outperf_1_month:.2%}")
print(f"Probability of UMD outperforming the market in 15 years: {prob_outperf_15_year:.2%}")

Probability of UMD outperforming the market in a single month: 43.45%
Probability of UMD outperforming the market in 15 years: 1.35%


In [145]:
umd_mkt_spread.values == X.values

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [147]:
umd_mkt_spread

date
2009-01-01    0.0626
2009-02-01    0.1496
2009-03-01   -0.2121
2009-04-01   -0.5170
2009-05-01   -0.1842
               ...  
2024-04-01    0.0436
2024-05-01   -0.0427
2024-06-01   -0.0184
2024-07-01   -0.0368
2024-08-01    0.0307
Length: 188, dtype: float64

In [151]:
factor_data

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.0551,0.0175,-0.0170,0.0755
1980-02-01,-0.0122,0.0061,0.0004,0.0788
1980-03-01,-0.1290,-0.0101,0.0146,-0.0955
1980-04-01,0.0397,0.0106,-0.0210,-0.0043
1980-05-01,0.0526,0.0038,0.0034,-0.0112
...,...,...,...,...
2024-04-01,-0.0467,-0.0052,0.0148,-0.0042
2024-05-01,0.0434,-0.0166,0.0298,-0.0002
2024-06-01,0.0277,-0.0331,0.0051,0.0090
